In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 출력을 위한 글꼴 설정
plt.rcParams['axes.unicode_minus'] = False    # 문자 - (마이너스) 정상 출력을 위한 코드

# 컨텐츠기반 추천시스템을 위한 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#실수 설정 옵션 (소수 셋째 짜리까지)
pd.options.display.float_format = '{:.3f}'.format

## 데이터셋

In [2]:
df_closet=pd.read_excel('C:/Users/82102/jupi python/프리온보딩/기술과제 제출/클로젯셰어_과제DB 대체 자료.xlsx')

In [3]:
df_closet.head()

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,NaN
2,Danha,embroidery point oriental mini skirt blue,a1140859,2021-01-02,2021-01-15,4days,B,NaN
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,NaN
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y


## 결측치 

In [4]:
#app_yn n 채우자
df_closet["app_yn"].fillna('N', inplace = True)
df_closet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   brandnm     1442 non-null   object        
 1   goodsnm     1471 non-null   object        
 2   USER        1471 non-null   object        
 3   reg_date    1466 non-null   datetime64[ns]
 4   order_date  1471 non-null   datetime64[ns]
 5   p_type      1471 non-null   object        
 6   sub_type    1471 non-null   object        
 7   app_yn      1471 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 92.1+ KB


In [5]:
# brandnm, reg_date 결측치 확인
brandnm_nl=df_closet.loc[df_closet['brandnm'].isnull()]
reg_date_nl = df_closet.loc[df_closet['reg_date'].isnull()]

In [6]:
# reg_date 결측치
# 유저, 브랜드, 다른 피쳐가 거의 다르다 
reg_date_nl

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
265,Gucci,mamont chain bag red,a1144008,NaT,2021-01-21,7days,BA,Y
351,Lanvin,banding point jumper skyblue,a1142855,NaT,2021-01-18,7days,O,N
564,Gucci,leather point cross bag,a1146490,NaT,2021-02-18,4days,BA,Y
746,Fendi,monogram zucchino shoulder bag,a1145478,NaT,2021-02-04,4days,BA,N
1254,Fendi,back to school bag 1,a1149094,NaT,2021-03-12,7days,BA,Y


In [7]:
#결측값 정리 파일
df_tp=pd.read_csv('TP.csv')

In [8]:
df_tp= df_tp.rename(columns={
    '브랜드 결측값 브랜드 후보':'brandnm'
})

In [9]:
df_tp_dict=df_tp.to_dict()

In [10]:
for i in  list(df_tp_dict.values()):
    df_closet['brandnm'] = df_closet['brandnm'].fillna(i)

In [11]:
df_closet.replace(' ', np.nan, inplace = True)

In [12]:
df_closet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   brandnm     1469 non-null   object        
 1   goodsnm     1471 non-null   object        
 2   USER        1471 non-null   object        
 3   reg_date    1466 non-null   datetime64[ns]
 4   order_date  1471 non-null   datetime64[ns]
 5   p_type      1471 non-null   object        
 6   sub_type    1471 non-null   object        
 7   app_yn      1471 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 92.1+ KB


## 주문 sub_type 별로 컷

In [13]:
df_closet_ba = df_closet[df_closet['sub_type']=='BA']
df_closet_o = df_closet[df_closet['sub_type']=='O']
df_closet_d = df_closet[df_closet['sub_type']=='D']
df_closet_t = df_closet[df_closet['sub_type']=='T']
df_closet_b = df_closet[df_closet['sub_type']=='B']
df_closet_tp = df_closet[df_closet['sub_type']=='TP']

In [14]:
df_closet_b_list = df_closet_b['goodsnm'].to_list()

In [15]:
df_closet_b_list

['embroidery point oriental mini skirt blue',
 'pocket basic pattern slacks black',
 'slit basic pattern skirt white',
 'check pattern unbalance skirt ivory',
 'basic pattern skirt grey',
 'heart pattern quilted skirts',
 'pocket basic pattern wide-pants navy',
 'roll up basic pattern slacks navy',
 'basic pattern unbalance skirt skyblue',
 'pocket graphics pattern sweat-pants white',
 'pocket basic pattern wool slacks grey',
 'pocket basic pattern slacks navy',
 'pocket basic pattern skirt blue',
 'button check pattern pants',
 'pleats pocket wool skirt brown',
 'pleats basic pattern pleats-skirt grey',
 'basic pattern skirt grey',
 'quilting detail denim skirt',
 'pleats basic pattern wool pleats-skirt brown',
 'pattern pointed a-line skirts',
 'pocket basic pattern skirt white',
 'button basic pattern pants brown',
 'slit basic pattern skirt black',
 'paisley print pattern skirt',
 'pocket check pattern slacks charcoal',
 'pocket stripe pattern slacks black',
 'button check pattern 

In [18]:
#goodsnm 벡터화
tfidf = TfidfVectorizer()

tfidf_matrix_bottom = tfidf.fit_transform(df_closet_b['goodsnm'])
print(tfidf_matrix_bottom.shape)

(58, 69)


In [19]:
print(tfidf_matrix_bottom)

  (0, 5)	0.39048674831878877
  (0, 54)	0.15804822793757034
  (0, 33)	0.4638129375111069
  (0, 36)	0.4638129375111069
  (0, 44)	0.4638129375111069
  (0, 16)	0.4209198665128158
  (1, 4)	0.5759963158662922
  (1, 57)	0.5120829294138277
  (1, 39)	0.25893853616766477
  (1, 2)	0.33817984354418973
  (1, 43)	0.4739035191040322
  (2, 65)	0.6129966546870037
  (2, 58)	0.6470106706227533
  (2, 39)	0.21743905318077447
  (2, 2)	0.28398053867678286
  (2, 54)	0.27872496344520375
  (3, 25)	0.5848835771774631
  (3, 63)	0.5848835771774631
  (3, 10)	0.47505325893941164
  (3, 39)	0.18467769224081143
  (3, 54)	0.23672970547829816
  (4, 23)	0.7899825583617416
  (4, 39)	0.29401202353489014
  (4, 2)	0.3839866463706227
  (4, 54)	0.3768802766266753
  :	:
  (53, 26)	0.6841117393707405
  (53, 39)	0.18185929214251664
  (53, 2)	0.23751253047939874
  (53, 43)	0.3328348101585862
  (53, 5)	0.5759575617426456
  (54, 35)	0.6481398792132462
  (54, 57)	0.47704565243440006
  (54, 39)	0.24122167686376425
  (54, 2)	0.315041206

In [20]:
#코사인 유사도 측정
cosine_matrix_bottom = cosine_similarity(tfidf_matrix_bottom, tfidf_matrix_bottom)

In [21]:
cosine_matrix_bottom.shape

(58, 58)

In [23]:
np.round(cosine_matrix_bottom, 4)

array([[1.    , 0.    , 0.0441, ..., 0.0441, 0.0331, 0.0362],
       [0.    , 1.    , 0.1523, ..., 0.1523, 0.2091, 0.0462],
       [0.0441, 0.1523, 1.    , ..., 1.    , 0.0583, 0.1026],
       ...,
       [0.0441, 0.1523, 1.    , ..., 1.    , 0.0583, 0.1026],
       [0.0331, 0.2091, 0.0583, ..., 0.0583, 1.    , 0.0479],
       [0.0362, 0.0462, 0.1026, ..., 0.1026, 0.0479, 1.    ]])

In [24]:
#딕셔너리 생성
goods_id = {}
for i, c in enumerate(df_closet_b['goodsnm']): goods_id[i] = c
#아이디 별 굿즈 이름
id2goods_b = {}
for i, c in goods_id.items(): id2goods_b[c] =i

In [26]:
idx = id2goods_b['pocket basic pattern slacks black']
sim_scores_b = [(i,c) for i, c in enumerate(cosine_matrix_bottom[idx]) if i !=idx] 
sim_scores_b = sorted(sim_scores_b, key =lambda x: x[1], reverse = True)
sim_scores_b[0:10]

[(1, 1.0000000000000002),
 (48, 1.0000000000000002),
 (25, 0.6538873230543492),
 (11, 0.6225073330421497),
 (52, 0.6225073330421497),
 (54, 0.6225073330421497),
 (50, 0.6214580547261949),
 (31, 0.5905248692538959),
 (10, 0.5223808591514305),
 (30, 0.48766280518676697)]

In [29]:
sim_scores_b = [(goods_id[i], score) for i , score in sim_scores_b[0:10]]
sim_scores_b

[('pocket basic pattern slacks black', 1.0000000000000002),
 ('pocket basic pattern slacks black', 1.0000000000000002),
 ('pocket stripe pattern slacks black', 0.6538873230543492),
 ('pocket basic pattern slacks navy', 0.6225073330421497),
 ('pocket basic pattern slacks navy', 0.6225073330421497),
 ('pocket basic pattern slacks navy', 0.6225073330421497),
 ('ribbon basic pattern slacks black', 0.6214580547261949),
 ('buckle basic pattern slacks black', 0.5905248692538959),
 ('pocket basic pattern wool slacks grey', 0.5223808591514305),
 ('basic pattern wool skirt black', 0.48766280518676697)]

In [30]:
#다른 후보
idx2 = id2goods_b['button check pattern wool slacks brown']
sim_scores_b = [(i,c) for i, c in enumerate(cosine_matrix_bottom[idx2]) if i !=idx2] 
sim_scores_b = sorted(sim_scores_b, key =lambda x: x[1], reverse = True)
sim_scores_b[0:10]

[(21, 0.585255072653696),
 (13, 0.5755928906555773),
 (14, 0.4646411077909181),
 (57, 0.45864558302731673),
 (10, 0.40518533826161973),
 (18, 0.35994463017234096),
 (27, 0.35757640836705484),
 (24, 0.35306697132206044),
 (30, 0.33373512630582247),
 (3, 0.23211925019507262)]

In [31]:
sim_scores_b2 = [(goods_id[i], score) for i , score in sim_scores_b[0:10]]
sim_scores_b2

[('button basic pattern pants brown', 0.585255072653696),
 ('button check pattern pants', 0.5755928906555773),
 ('pleats pocket wool skirt brown', 0.4646411077909181),
 ('check pattern kilt-skirt brown', 0.45864558302731673),
 ('pocket basic pattern wool slacks grey', 0.40518533826161973),
 ('pleats basic pattern wool pleats-skirt brown', 0.35994463017234096),
 ('pocket basic pattern wool slacks cream', 0.35757640836705484),
 ('pocket check pattern slacks charcoal', 0.35306697132206044),
 ('basic pattern wool skirt black', 0.33373512630582247),
 ('check pattern unbalance skirt ivory', 0.23211925019507262)]